# DES Simulation

This notebook generates the results used in the report for the second assignment of the course 'Stochastic Simulation'.

Authors:

1. Divyaben Hasmukhbhai Gajera
- Affiliation: MSc Computational Science, University of Amsterdam
- Email: divya.gajera@student.uva.nl
- Student Number: 14932644

2. K. López
- Affiliation: MSc Computational Science, University of Amsterdam
- Email: kenia.lopez.sotomayor@student.uva.nl
- Student Number: 12965081

3. T. P. Glansdorp
- Affiliation: MSc Computational Science, University of Amsterdam
- Email: thomas.glansdorp@student.uva.nl
- Student Number: 12748587

In [ ]:
import simpy as sp
import numpy as np
import matplotlib.pyplot as plt

## DES program that implements FIFO scheduling case

### Statistical significance 

### Numer of measurements required

## DES program that implements shortest job first case for M/M/1

## Different service rate distribution on M/D/n and M/D/1

## Long-tail distribution